# 整理PowerBI所需資料集

In [4]:
# 整理PowerBIn所需資料集
# 2021/05/22 蘇彥庭
import json
import pandas as pd
from sqlalchemy import create_engine

# 設定pandas呈現表格時所有欄位都要呈現出來
pd.set_option('display.max_columns', None)

In [5]:
# 建立連線函數
def CreateDBEngine(secretFileName='dbSecret.json'):
    secretFile = json.load(open(secretFileName, 'r'))
    host = secretFile['host']
    username = secretFile['user']
    password = secretFile['password']
    port = secretFile['port']
    database = secretFile['dbName']
    return create_engine(f'mysql+mysqlconnector://{username}:{password}@{host}:{port}/{database}', echo=False)

# 整理SEGIS資料
將村里層級的資料彙整為鄉鎮市區層級

In [15]:
# 查詢內政部大數據鄉鎮市區資料指令
sqlQuery = '''
select
convert(replace(INFO_TIME, 'Y', ''), unsigned) as year,  # 年度
COUNTY_ID as county_id,  # 縣市代碼
COUNTY as county,  # 縣市名稱
TOWN_ID as town_id,  # 鄉鎮市區代碼
TOWN as town,  # 鄉鎮市區名稱

# 人口結構
sum(COLUMN2) as population,  # 人口數
sum(COLUMN52) as births_nums,  # 出生人口數
sum(COLUMN59) as deaths_nums,  # 死亡人口數
sum(COLUMN66) as immigration_nums,  # 遷入人口數
sum(COLUMN67) as migration_nums,  # 遷出人口數
sum(COLUMN33) as toddler_nums, # 幼年人口數
sum(COLUMN34) as worker_nums,  # 青壯年人口數
sum(COLUMN35) as elderly_nums,  # 老年人口數

# 婚育概況
sum(COLUMN60) as married_pairs,  # 結婚對數

# 托育教保
sum(COLUMN108+COLUMN109+COLUMN110) as age_0_2_population,  # 0-2歲學齡人口數
sum(COLUMN111+COLUMN112+COLUMN113+COLUMN114) as age_3_6_population  # 3-6學齡人口數

from project.demographic
where info_time <= 108  # SEGIS的109年資料有問題故不使用
group by info_time, county_id, county, town_id, town
order by COUNTY_ID, TOWN_ID, info_time;
'''

# 取得資料
segisData = pd.read_sql(sql=sqlQuery, con=CreateDBEngine())
segisData.head(10)

,year,county_id,county,town_id,town,population,births_nums,deaths_nums,immigration_nums,migration_nums,toddler_nums,worker_nums,elderly_nums,married_pairs,age_0_2_population,age_3_6_population
0,104,10002,宜蘭縣,10002010,宜蘭市,96051.0,732.0,680.0,3926.0,3812.0,13808.0,69774.0,12469.0,543.0,2007.0,3239.0
1,105,10002,宜蘭縣,10002010,宜蘭市,95907.0,746.0,658.0,3563.0,3795.0,13556.0,69298.0,13053.0,511.0,2080.0,3165.0
2,106,10002,宜蘭縣,10002010,宜蘭市,95918.0,705.0,711.0,3760.0,3743.0,13387.0,68989.0,13542.0,502.0,2208.0,3214.0
3,107,10002,宜蘭縣,10002010,宜蘭市,95732.0,586.0,671.0,3687.0,3788.0,13165.0,68421.0,14146.0,508.0,2107.0,3423.0
4,108,10002,宜蘭縣,10002010,宜蘭市,95562.0,592.0,730.0,3615.0,3647.0,12918.0,67972.0,14672.0,536.0,1929.0,3459.0
5,104,10002,宜蘭縣,10002020,羅東鎮,72404.0,559.0,460.0,3524.0,3752.0,12101.0,50956.0,9347.0,448.0,1764.0,2822.0
6,105,10002,宜蘭縣,10002020,羅東鎮,72485.0,595.0,501.0,3366.0,3379.0,12067.0,50574.0,9844.0,426.0,1721.0,2918.0
7,106,10002,宜蘭縣,10002020,羅東鎮,72319.0,552.0,504.0,3192.0,3406.0,11912.0,50054.0,10353.0,395.0,1689.0,3015.0
8,107,10002,宜蘭縣,10002020,羅東鎮,72433.0,494.0,524.0,3389.0,3245.0,11952.0,49628.0,10853.0,392.0,1657.0,3011.0
9,108,10002,宜蘭縣,10002020,羅東鎮,71898.0,395.0,508.0,2973.0,3395.0,11614.0,48963.0,11321.0,358.0,1491.0,2918.0


In [16]:
# 為能夠對應其他來源資料 鄉鎮市區代碼若只有7碼 則第一碼補0
# 此為108年內政大數據資料應用組競賽用資料集_村里資料問題
reviseIdx = segisData['town_id'].str.len() == 7
segisData.loc[reviseIdx, 'town_id'] = '0' + segisData.loc[reviseIdx, 'town_id']

In [17]:
# 判斷是否有缺值資料
segisData[segisData.isnull().any(axis=1)]

,year,county_id,county,town_id,town,population,births_nums,deaths_nums,immigration_nums,migration_nums,toddler_nums,worker_nums,elderly_nums,married_pairs,age_0_2_population,age_3_6_population


In [18]:
# 查詢郵政編碼資料 以郵政編碼來排序 讓PowerBI選項可以符合使用者直覺
sqlQuery = '''
select countyname as county, townname as town, zipcode FROM project.postal_code order by zipcode;
'''

# 取得資料
zipData = pd.read_sql(sql=sqlQuery, con=CreateDBEngine())
zipData.head(10)

,county,town,zipcode
0,臺北市,中正區,100
1,臺北市,大同區,103
2,臺北市,中山區,104
3,臺北市,松山區,105
4,臺北市,大安區,106
5,臺北市,萬華區,108
6,臺北市,信義區,110
7,臺北市,士林區,111
8,臺北市,北投區,112
9,臺北市,內湖區,114


In [19]:
# 併表並重新排序
segisData = pd.merge(segisData, zipData, how='left', on=['county', 'town'])
segisData = segisData.sort_values(by=['zipcode', 'year'])
segisData

,year,county_id,county,town_id,town,population,births_nums,deaths_nums,immigration_nums,migration_nums,toddler_nums,worker_nums,elderly_nums,married_pairs,age_0_2_population,age_3_6_population,zipcode
1020,104,63000,臺北市,63000050,中正區,162475.0,1679.0,1112.0,10348.0,11828.0,26432.0,110438.0,25605.0,1104.0,5424.0,6877.0,100
1021,105,63000,臺北市,63000050,中正區,160403.0,1608.0,1109.0,9774.0,12345.0,25951.0,108042.0,26410.0,948.0,5287.0,7169.0,100
1022,106,63000,臺北市,63000050,中正區,159608.0,1466.0,1050.0,10315.0,11526.0,25854.0,106395.0,27359.0,791.0,5195.0,7317.0,100
1023,107,63000,臺北市,63000050,中正區,159000.0,1333.0,1082.0,10750.0,11609.0,25802.0,104916.0,28282.0,745.0,4800.0,7583.0,100
1024,108,63000,臺北市,63000050,中正區,158014.0,1235.0,1074.0,11214.0,12361.0,25634.0,103238.0,29142.0,745.0,4381.0,7500.0,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
890,104,10015,花蓮縣,10015100,富里鄉,10633.0,76.0,151.0,304.0,445.0,1103.0,7241.0,2289.0,40.0,214.0,238.0,983
891,105,10015,花蓮縣,10015100,富里鄉,10557.0,57.0,162.0,389.0,360.0,1070.0,7167.0,2320.0,44.0,187.0,264.0,983
892,106,10015,花蓮縣,10015100,富里鄉,10413.0,55.0,192.0,347.0,354.0,1010.0,7058.0,2345.0,45.0,188.0,238.0,983
893,107,10015,花蓮縣,10015100,富里鄉,10277.0,59.0,167.0,400.0,428.0,917.0,6991.0,2369.0,37.0,165.0,234.0,983


In [20]:
# 判斷是否有缺值資料
segisData[segisData.isnull().any(axis=1)]

,year,county_id,county,town_id,town,population,births_nums,deaths_nums,immigration_nums,migration_nums,toddler_nums,worker_nums,elderly_nums,married_pairs,age_0_2_population,age_3_6_population,zipcode


# 整理縣市層級資料

In [86]:
# 建立主表
countyData = segisData[['year', 'county']].drop_duplicates().reset_index(drop=True)
countyData

,year,county
0,104,臺北市
1,105,臺北市
2,106,臺北市
3,107,臺北市
4,108,臺北市
...,...,...
105,104,花蓮縣
106,105,花蓮縣
107,106,花蓮縣
108,107,花蓮縣


In [87]:
# 查詢可支配所得資料
sqlQuery = '''
select 
year,  # 年份
county,  # 縣市
disposable_income_median  # 可支配所得
from project.family_income_exp
where county != '總平均';
'''

# 取得資料
disposableIncomeData = pd.read_sql(sql=sqlQuery, con=CreateDBEngine())
disposableIncomeData

,year,county,disposable_income_median
0,101,新北市,828521.0
1,101,臺北市,1108547.0
2,101,臺中市,798775.0
3,101,臺南市,689633.0
4,101,高雄市,795806.0
...,...,...,...
155,108,花蓮縣,630243.0
156,108,澎湖縣,716494.0
157,108,基隆市,843101.0
158,108,新竹市,1097946.0


In [88]:
# 併表
countyData = pd.merge(countyData, disposableIncomeData, how='left', on=['year', 'county'])
countyData

,year,county,disposable_income_median
0,104,臺北市,1133742.0
1,105,臺北市,1113565.0
2,106,臺北市,1128510.0
3,107,臺北市,1181115.0
4,108,臺北市,1243663.0
...,...,...,...
105,104,花蓮縣,602157.0
106,105,花蓮縣,588202.0
107,106,花蓮縣,621922.0
108,107,花蓮縣,649983.0


In [89]:
# 查詢失業率資料
sqlQuery = '''
select year, area as county, total as unemployment_rate from project.unemployment_rate;
'''

# 取得資料
unemploymentRateData = pd.read_sql(sql=sqlQuery, con=CreateDBEngine())
unemploymentRateData

,year,county,unemployment_rate
0,104,臺灣地區,3.78
1,104,北部區域,3.80
2,104,新北市,3.70
3,104,臺北市,3.80
4,104,桃園市,3.90
...,...,...,...
145,109,屏東縣,3.80
146,109,澎湖縣,3.90
147,109,東部區域,3.70
148,109,臺東縣,3.70


In [90]:
# 併表
countyData = pd.merge(countyData, unemploymentRateData, how='left', on=['year', 'county'])
countyData

,year,county,disposable_income_median,unemployment_rate
0,104,臺北市,1133742.0,3.8
1,105,臺北市,1113565.0,3.9
2,106,臺北市,1128510.0,3.8
3,107,臺北市,1181115.0,3.7
4,108,臺北市,1243663.0,3.7
...,...,...,...,...
105,104,花蓮縣,602157.0,3.7
106,105,花蓮縣,588202.0,3.9
107,106,花蓮縣,621922.0,3.7
108,107,花蓮縣,649983.0,3.5


In [91]:
# 查詢房價負擔能力統計資料
sqlQuery = '''
select year, 
city as county, 
mbr as mortgageBurdenRatio,  # 房價負擔率(%)
pir as priceIncomeRatio  # 房價所得比(倍)
from project.house_burden_data where season = 4;
'''

# 取得資料
housingAffordabilityData = pd.read_sql(sql=sqlQuery, con=CreateDBEngine())
housingAffordabilityData

,year,county,mortgageBurdenRatio,priceIncomeRatio
0,109,全國,36.81,9.20
1,108,全國,35.15,8.58
2,107,全國,35.12,8.57
3,106,全國,37.58,9.16
4,105,全國,38.34,9.32
...,...,...,...,...
205,104,嘉義市,23.73,5.64
206,103,嘉義市,25.55,6.03
207,102,嘉義市,25.55,6.05
208,101,嘉義市,22.60,5.37


In [92]:
# 併表
countyData = pd.merge(countyData, housingAffordabilityData, how='left', on=['year', 'county'])
countyData

,year,county,disposable_income_median,unemployment_rate,mortgageBurdenRatio,priceIncomeRatio
0,104,臺北市,1133742.0,3.8,66.26,15.75
1,105,臺北市,1113565.0,3.9,62.48,15.18
2,106,臺北市,1128510.0,3.8,61.52,14.99
3,107,臺北市,1181115.0,3.7,56.83,13.86
4,108,臺北市,1243663.0,3.7,57.11,13.94
...,...,...,...,...,...,...
105,104,花蓮縣,602157.0,3.7,30.27,7.19
106,105,花蓮縣,588202.0,3.9,33.47,8.13
107,106,花蓮縣,621922.0,3.7,32.86,8.01
108,107,花蓮縣,649983.0,3.5,34.63,8.45


In [93]:
# 查詢女性勞動力參與率
sqlQuery = '''
select 
year,
area as county,
age_15_19_female as female_labor_ratio_15_19,  # 15-19歲女性勞動力參與率(%)
age_20_24_female as female_labor_ratio_20_24,  # 20-24歲女性勞動力參與率(%)
age_25_29_female as female_labor_ratio_25_29,  # 25-29歲女性勞動力參與率(%)
age_30_34_female as female_labor_ratio_30_34,  # 30-34歲女性勞動力參與率(%)
age_35_39_female as female_labor_ratio_35_39,  # 35-39歲女性勞動力參與率(%)
age_40_44_female as female_labor_ratio_40_44   # 40-44歲女性勞動力參與率(%)
from project.labor_force_participation_rate;
'''

# 取得資料
laborForceData = pd.read_sql(sql=sqlQuery, con=CreateDBEngine())
laborForceData

,year,county,female_labor_ratio_15_19,female_labor_ratio_20_24,female_labor_ratio_25_29,female_labor_ratio_30_34,female_labor_ratio_35_39,female_labor_ratio_40_44
0,102,臺灣地區,7.77,53.92,90.3,79.13,74.59,73.82
1,102,北部地區,7.90,54.00,90.2,79.90,75.00,73.70
2,102,新北市,8.90,55.10,91.0,79.80,75.90,72.00
3,102,臺北市,6.40,48.60,88.9,81.60,75.50,76.90
4,102,基隆市,7.70,51.40,92.4,79.10,75.20,72.50
...,...,...,...,...,...,...,...,...
195,109,屏東縣,9.70,65.40,91.4,88.20,81.60,73.80
196,109,澎湖縣,5.60,49.70,88.2,77.70,71.30,68.20
197,109,東部地區,7.80,54.50,93.0,87.00,83.50,74.60
198,109,臺東縣,3.90,52.60,91.9,86.40,85.20,78.00


In [94]:
# 併表
countyData = pd.merge(countyData, laborForceData, how='left', on=['year', 'county'])
countyData

,year,county,disposable_income_median,unemployment_rate,mortgageBurdenRatio,priceIncomeRatio,female_labor_ratio_15_19,female_labor_ratio_20_24,female_labor_ratio_25_29,female_labor_ratio_30_34,female_labor_ratio_35_39,female_labor_ratio_40_44
0,104,臺北市,1133742.0,3.8,66.26,15.75,7.4,48.1,89.9,83.5,75.3,78.2
1,105,臺北市,1113565.0,3.9,62.48,15.18,5.7,48.1,91.3,86.8,78.7,76.4
2,106,臺北市,1128510.0,3.8,61.52,14.99,6.9,52.4,89.2,85.7,81.9,78.8
3,107,臺北市,1181115.0,3.7,56.83,13.86,10.2,56.3,89.5,90.1,81.4,81.1
4,108,臺北市,1243663.0,3.7,57.11,13.94,7.8,60.3,90.3,87.8,85.4,79.9
...,...,...,...,...,...,...,...,...,...,...,...,...
105,104,花蓮縣,602157.0,3.7,30.27,7.19,6.5,50.1,87.0,82.3,68.9,73.0
106,105,花蓮縣,588202.0,3.9,33.47,8.13,7.2,48.5,87.1,75.3,74.1,76.7
107,106,花蓮縣,621922.0,3.7,32.86,8.01,3.3,53.3,89.7,78.8,73.8,78.3
108,107,花蓮縣,649983.0,3.5,34.63,8.45,7.0,49.0,91.7,85.2,74.8,74.4


In [95]:
# 查詢女性生育年齡
sqlQuery = '''
select year, county,
round(mean_age_childbearing, 1) as mean_age_childbearing,
round(mean_age_first_childbearing, 1) as mean_age_first_childbearing
from project.fertility_rate;
'''

# 取得資料
fertilityRateData = pd.read_sql(sql=sqlQuery, con=CreateDBEngine())
fertilityRateData

,year,county,mean_age_childbearing,mean_age_first_childbearing
0,100,總計,30.9,29.9
1,101,總計,31.1,30.1
2,102,總計,31.4,30.4
3,103,總計,31.5,30.5
4,104,總計,31.7,30.6
...,...,...,...,...
202,104,連江縣,32.1,30.4
203,105,連江縣,31.9,30.2
204,106,連江縣,30.9,29.5
205,107,連江縣,32.3,31.2


In [96]:
# 併表
countyData = pd.merge(countyData, fertilityRateData, how='left', on=['year', 'county'])
countyData

,year,county,disposable_income_median,unemployment_rate,mortgageBurdenRatio,priceIncomeRatio,female_labor_ratio_15_19,female_labor_ratio_20_24,female_labor_ratio_25_29,female_labor_ratio_30_34,female_labor_ratio_35_39,female_labor_ratio_40_44,mean_age_childbearing,mean_age_first_childbearing
0,104,臺北市,1133742.0,3.8,66.26,15.75,7.4,48.1,89.9,83.5,75.3,78.2,33.3,32.4
1,105,臺北市,1113565.0,3.9,62.48,15.18,5.7,48.1,91.3,86.8,78.7,76.4,33.5,32.5
2,106,臺北市,1128510.0,3.8,61.52,14.99,6.9,52.4,89.2,85.7,81.9,78.8,33.7,32.7
3,107,臺北市,1181115.0,3.7,56.83,13.86,10.2,56.3,89.5,90.1,81.4,81.1,33.8,32.9
4,108,臺北市,1243663.0,3.7,57.11,13.94,7.8,60.3,90.3,87.8,85.4,79.9,33.9,33.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,104,花蓮縣,602157.0,3.7,30.27,7.19,6.5,50.1,87.0,82.3,68.9,73.0,30.0,28.6
106,105,花蓮縣,588202.0,3.9,33.47,8.13,7.2,48.5,87.1,75.3,74.1,76.7,30.3,28.8
107,106,花蓮縣,621922.0,3.7,32.86,8.01,3.3,53.3,89.7,78.8,73.8,78.3,30.4,28.7
108,107,花蓮縣,649983.0,3.5,34.63,8.45,7.0,49.0,91.7,85.2,74.8,74.4,30.2,28.6


In [97]:
# 併入房屋自有率
sqlQuery = '''
select year, county,
round(home_ownership_rate, 2) as home_ownership_rate
from project.home_ownership_rate;
'''

# 取得資料
houseOwnerRatioData = pd.read_sql(sql=sqlQuery, con=CreateDBEngine())
houseOwnerRatioData

,year,county,home_ownership_rate
0,100,總平均,84.58
1,100,新北市,83.84
2,100,臺北市,81.94
3,100,臺中市,81.30
4,100,臺南市,82.67
...,...,...,...
184,108,花蓮縣,85.48
185,108,澎湖縣,86.97
186,108,基隆市,87.46
187,108,新竹市,83.73


In [98]:
# 併表
countyData = pd.merge(countyData, houseOwnerRatioData, how='left', on=['year', 'county'])
countyData

,year,county,disposable_income_median,unemployment_rate,mortgageBurdenRatio,priceIncomeRatio,female_labor_ratio_15_19,female_labor_ratio_20_24,female_labor_ratio_25_29,female_labor_ratio_30_34,female_labor_ratio_35_39,female_labor_ratio_40_44,mean_age_childbearing,mean_age_first_childbearing,home_ownership_rate
0,104,臺北市,1133742.0,3.8,66.26,15.75,7.4,48.1,89.9,83.5,75.3,78.2,33.3,32.4,82.54
1,105,臺北市,1113565.0,3.9,62.48,15.18,5.7,48.1,91.3,86.8,78.7,76.4,33.5,32.5,84.16
2,106,臺北市,1128510.0,3.8,61.52,14.99,6.9,52.4,89.2,85.7,81.9,78.8,33.7,32.7,83.69
3,107,臺北市,1181115.0,3.7,56.83,13.86,10.2,56.3,89.5,90.1,81.4,81.1,33.8,32.9,83.40
4,108,臺北市,1243663.0,3.7,57.11,13.94,7.8,60.3,90.3,87.8,85.4,79.9,33.9,33.0,84.02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,104,花蓮縣,602157.0,3.7,30.27,7.19,6.5,50.1,87.0,82.3,68.9,73.0,30.0,28.6,82.62
106,105,花蓮縣,588202.0,3.9,33.47,8.13,7.2,48.5,87.1,75.3,74.1,76.7,30.3,28.8,84.92
107,106,花蓮縣,621922.0,3.7,32.86,8.01,3.3,53.3,89.7,78.8,73.8,78.3,30.4,28.7,80.69
108,107,花蓮縣,649983.0,3.5,34.63,8.45,7.0,49.0,91.7,85.2,74.8,74.4,30.2,28.6,84.96


In [99]:
# 併入幼兒園概況
sqlQuery = '''
select * from project.kindergarten_data;
'''

# 取得資料
kindergartenData = pd.read_sql(sql=sqlQuery, con=CreateDBEngine())
kindergartenData

,year,county,total_school_nums,public_school_nums,total_children,public_children
0,102,總計,6560.0,1919.0,448189.0,131910.0
1,102,臺灣地區,6528.0,1895.0,446212.0,130117.0
2,102,新北市,1147.0,242.0,71506.0,21354.0
3,102,臺北市,708.0,148.0,44656.0,15691.0
4,102,臺中市,677.0,149.0,57917.0,13668.0
...,...,...,...,...,...,...
195,109,新竹市,163.0,28.0,14728.0,2481.0
196,109,嘉義市,70.0,16.0,7237.0,2056.0
197,109,金馬地區,30.0,24.0,2495.0,2211.0
198,109,金門縣,25.0,19.0,2202.0,1918.0


In [100]:
# 併表
countyData = pd.merge(countyData, kindergartenData, how='left', on=['year', 'county'])
countyData

,year,county,disposable_income_median,unemployment_rate,mortgageBurdenRatio,priceIncomeRatio,female_labor_ratio_15_19,female_labor_ratio_20_24,female_labor_ratio_25_29,female_labor_ratio_30_34,female_labor_ratio_35_39,female_labor_ratio_40_44,mean_age_childbearing,mean_age_first_childbearing,home_ownership_rate,total_school_nums,public_school_nums,total_children,public_children
0,104,臺北市,1133742.0,3.8,66.26,15.75,7.4,48.1,89.9,83.5,75.3,78.2,33.3,32.4,82.54,687.0,149.0,47269.0,17863.0
1,105,臺北市,1113565.0,3.9,62.48,15.18,5.7,48.1,91.3,86.8,78.7,76.4,33.5,32.5,84.16,682.0,149.0,50079.0,18412.0
2,106,臺北市,1128510.0,3.8,61.52,14.99,6.9,52.4,89.2,85.7,81.9,78.8,33.7,32.7,83.69,689.0,150.0,52253.0,18950.0
3,107,臺北市,1181115.0,3.7,56.83,13.86,10.2,56.3,89.5,90.1,81.4,81.1,33.8,32.9,83.40,697.0,150.0,54229.0,19475.0
4,108,臺北市,1243663.0,3.7,57.11,13.94,7.8,60.3,90.3,87.8,85.4,79.9,33.9,33.0,84.02,694.0,150.0,56635.0,19980.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,104,花蓮縣,602157.0,3.7,30.27,7.19,6.5,50.1,87.0,82.3,68.9,73.0,30.0,28.6,82.62,131.0,85.0,6561.0,3523.0
106,105,花蓮縣,588202.0,3.9,33.47,8.13,7.2,48.5,87.1,75.3,74.1,76.7,30.3,28.8,84.92,130.0,85.0,6772.0,3632.0
107,106,花蓮縣,621922.0,3.7,32.86,8.01,3.3,53.3,89.7,78.8,73.8,78.3,30.4,28.7,80.69,128.0,85.0,7017.0,3769.0
108,107,花蓮縣,649983.0,3.5,34.63,8.45,7.0,49.0,91.7,85.2,74.8,74.4,30.2,28.6,84.96,129.0,85.0,7160.0,3870.0


In [101]:
# 併入托嬰中心資訊
sqlQuery = '''
select year-1911 as year, city as county, private_childcare_center, public_to_private_childcare_center
from project.number_of_childcare_centers_and_recipients;
'''

# 取得資料
childcareCentersData = pd.read_sql(sql=sqlQuery, con=CreateDBEngine())
childcareCentersData

,year,county,private_childcare_center,public_to_private_childcare_center
0,109,新北市,195,85
1,109,臺北市,177,75
2,109,桃園市,98,24
3,109,臺中市,155,14
4,109,臺南市,90,6
...,...,...,...,...
127,104,基隆市,3,2
128,104,新竹市,48,0
129,104,嘉義市,2,1
130,104,金門縣,0,1


In [102]:
# 併表
countyData = pd.merge(countyData, childcareCentersData, how='left', on=['year', 'county'])
countyData

,year,county,disposable_income_median,unemployment_rate,mortgageBurdenRatio,priceIncomeRatio,female_labor_ratio_15_19,female_labor_ratio_20_24,female_labor_ratio_25_29,female_labor_ratio_30_34,female_labor_ratio_35_39,female_labor_ratio_40_44,mean_age_childbearing,mean_age_first_childbearing,home_ownership_rate,total_school_nums,public_school_nums,total_children,public_children,private_childcare_center,public_to_private_childcare_center
0,104,臺北市,1133742.0,3.8,66.26,15.75,7.4,48.1,89.9,83.5,75.3,78.2,33.3,32.4,82.54,687.0,149.0,47269.0,17863.0,99,18
1,105,臺北市,1113565.0,3.9,62.48,15.18,5.7,48.1,91.3,86.8,78.7,76.4,33.5,32.5,84.16,682.0,149.0,50079.0,18412.0,114,18
2,106,臺北市,1128510.0,3.8,61.52,14.99,6.9,52.4,89.2,85.7,81.9,78.8,33.7,32.7,83.69,689.0,150.0,52253.0,18950.0,132,33
3,107,臺北市,1181115.0,3.7,56.83,13.86,10.2,56.3,89.5,90.1,81.4,81.1,33.8,32.9,83.40,697.0,150.0,54229.0,19475.0,149,70
4,108,臺北市,1243663.0,3.7,57.11,13.94,7.8,60.3,90.3,87.8,85.4,79.9,33.9,33.0,84.02,694.0,150.0,56635.0,19980.0,165,74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,104,花蓮縣,602157.0,3.7,30.27,7.19,6.5,50.1,87.0,82.3,68.9,73.0,30.0,28.6,82.62,131.0,85.0,6561.0,3523.0,5,0
106,105,花蓮縣,588202.0,3.9,33.47,8.13,7.2,48.5,87.1,75.3,74.1,76.7,30.3,28.8,84.92,130.0,85.0,6772.0,3632.0,4,0
107,106,花蓮縣,621922.0,3.7,32.86,8.01,3.3,53.3,89.7,78.8,73.8,78.3,30.4,28.7,80.69,128.0,85.0,7017.0,3769.0,4,0
108,107,花蓮縣,649983.0,3.5,34.63,8.45,7.0,49.0,91.7,85.2,74.8,74.4,30.2,28.6,84.96,129.0,85.0,7160.0,3870.0,4,0


In [103]:
# 確認是否有缺值
countyData[countyData.isnull().any(axis=1)]

,year,county,disposable_income_median,unemployment_rate,mortgageBurdenRatio,priceIncomeRatio,female_labor_ratio_15_19,female_labor_ratio_20_24,female_labor_ratio_25_29,female_labor_ratio_30_34,female_labor_ratio_35_39,female_labor_ratio_40_44,mean_age_childbearing,mean_age_first_childbearing,home_ownership_rate,total_school_nums,public_school_nums,total_children,public_children,private_childcare_center,public_to_private_childcare_center
15,104,連江縣,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.1,30.4,NaN,6.0,5.0,223.0,173.0,0,1
16,105,連江縣,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.9,30.2,NaN,5.0,5.0,292.0,292.0,0,1
17,106,連江縣,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.9,29.5,NaN,5.0,5.0,292.0,292.0,0,2
18,107,連江縣,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.3,31.2,NaN,5.0,5.0,286.0,286.0,0,2
19,108,連江縣,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.0,31.2,NaN,5.0,5.0,300.0,300.0,0,2
90,104,金門縣,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.5,30.5,NaN,24.0,19.0,1851.0,1675.0,0,1
91,105,金門縣,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.6,30.5,NaN,24.0,19.0,1936.0,1759.0,0,1
92,106,金門縣,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.8,30.3,NaN,24.0,19.0,2026.0,1844.0,0,1
93,107,金門縣,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.7,30.6,NaN,25.0,19.0,2037.0,1824.0,1,1
94,108,金門縣,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.2,31.5,NaN,25.0,19.0,2099.0,1872.0,1,1


# 輸出資料表

In [106]:
# 輸出資料表
countyData.to_csv("countyData.csv", encoding="utf-8", index=False)
segisData.to_csv("segisData.csv", encoding="utf-8", index=False)